# Module 12: Advanced Python Features

This module covers advanced Python features including descriptors, context managers, abstract base classes, type hints, and more.

## 1. Descriptors

### 1.1 Understanding Descriptors

In [ ]:
# Basic descriptor
class Descriptor:
    def __init__(self, name=None):
        self.name = name
    
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        print(f"Getting {self.name}")
        return obj.__dict__.get(self.name, None)
    
    def __set__(self, obj, value):
        print(f"Setting {self.name} to {value}")
        obj.__dict__[self.name] = value
    
    def __delete__(self, obj):
        print(f"Deleting {self.name}")
        del obj.__dict__[self.name]

class MyClass:
    attr = Descriptor('attr')
    
    def __init__(self):
        self.attr = 'initial value'

# Using descriptor
obj = MyClass()
print(f"Value: {obj.attr}")
obj.attr = 'new value'
del obj.attr

# Practical descriptor: Type checking
class TypedProperty:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type
    
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        return obj.__dict__.get(self.name)
    
    def __set__(self, obj, value):
        if not isinstance(value, self.expected_type):
            raise TypeError(f"{self.name} must be {self.expected_type.__name__}")
        obj.__dict__[self.name] = value
    
    def __delete__(self, obj):
        del obj.__dict__[self.name]

class Person:
    name = TypedProperty('name', str)
    age = TypedProperty('age', int)
    
    def __init__(self, name, age):
        self.name = name
        self.age = age

# Test type checking
person = Person("Alice", 30)
print(f"\nPerson: {person.name}, {person.age}")

try:
    person.age = "thirty"  # This will raise TypeError
except TypeError as e:
    print(f"Error: {e}")

### 1.2 Advanced Descriptors

In [ ]:
# Non-data descriptor (only __get__)
class LazyProperty:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__
    
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        
        # Calculate value and cache it
        value = self.func(obj)
        setattr(obj, self.name, value)
        return value

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @LazyProperty
    def area(self):
        print("Calculating area...")
        import math
        return math.pi * self.radius ** 2
    
    @LazyProperty
    def circumference(self):
        print("Calculating circumference...")
        import math
        return 2 * math.pi * self.radius

# Lazy evaluation
circle = Circle(5)
print(f"Area: {circle.area:.2f}")  # Calculates
print(f"Area again: {circle.area:.2f}")  # Uses cached value
print(f"Circumference: {circle.circumference:.2f}")

# Bounded descriptor with validation
class BoundedValue:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
    
    def __set_name__(self, owner, name):
        self.name = name
    
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        return obj.__dict__.get(self.name)
    
    def __set__(self, obj, value):
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"{self.name} must be >= {self.min_value}")
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self.name} must be <= {self.max_value}")
        obj.__dict__[self.name] = value

class Temperature:
    celsius = BoundedValue(-273.15, 1000)
    
    def __init__(self, celsius):
        self.celsius = celsius

# Test bounded values
temp = Temperature(25)
print(f"\nTemperature: {temp.celsius}°C")

try:
    temp.celsius = -300  # Below absolute zero
except ValueError as e:
    print(f"Error: {e}")

## 2. Advanced Context Managers

### 2.1 Custom Context Managers

In [ ]:
import time
from contextlib import contextmanager, ExitStack, suppress
import sys
from io import StringIO

# Class-based context manager
class Timer:
    def __init__(self, name="Operation"):
        self.name = name
        self.start_time = None
        self.elapsed = None
    
    def __enter__(self):
        self.start_time = time.time()
        print(f"Starting {self.name}...")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.elapsed = time.time() - self.start_time
        print(f"{self.name} took {self.elapsed:.4f} seconds")
        
        if exc_type:
            print(f"Exception occurred: {exc_value}")
        return False  # Don't suppress exceptions

# Using timer
with Timer("Calculation"):
    total = sum(range(1000000))

# Decorator-based context manager
@contextmanager
def managed_resource(name):
    print(f"Acquiring {name}")
    resource = {"name": name, "active": True}
    try:
        yield resource
    finally:
        resource["active"] = False
        print(f"Releasing {name}")

with managed_resource("Database Connection") as conn:
    print(f"Using {conn['name']}")

# Redirect stdout
@contextmanager
def capture_output():
    old_stdout = sys.stdout
    sys.stdout = StringIO()
    try:
        yield sys.stdout
    finally:
        sys.stdout = old_stdout

with capture_output() as output:
    print("This is captured")
    print("So is this")

captured_text = output.getvalue()
print(f"\nCaptured: {repr(captured_text)}")

### 2.2 Advanced Context Manager Patterns

In [ ]:
from contextlib import ExitStack, closing, redirect_stdout
import tempfile
import os

# ExitStack for dynamic context management
def process_files(filenames):
    with ExitStack() as stack:
        files = [
            stack.enter_context(open(fname, 'w'))
            for fname in filenames
        ]
        
        for i, f in enumerate(files):
            f.write(f"File {i} content\n")
        
        print(f"Processed {len(files)} files")

# Create temp files and process
temp_files = [tempfile.mktemp(suffix='.txt') for _ in range(3)]
process_files(temp_files)

# Clean up
for f in temp_files:
    if os.path.exists(f):
        os.unlink(f)

# Nested context managers
@contextmanager
def nested_contexts(*managers):
    with ExitStack() as stack:
        results = [stack.enter_context(mgr) for mgr in managers]
        yield results

# Using nested contexts
with nested_contexts(
    Timer("Operation 1"),
    managed_resource("Resource A"),
    managed_resource("Resource B")
) as (timer, res_a, res_b):
    print("Working with multiple resources")

# Reentrant context manager
class ReentrantLock:
    def __init__(self):
        self.count = 0
    
    def __enter__(self):
        self.count += 1
        print(f"Lock acquired (depth: {self.count})")
        return self
    
    def __exit__(self, *args):
        self.count -= 1
        print(f"Lock released (depth: {self.count})")

lock = ReentrantLock()

with lock:
    print("Outer context")
    with lock:
        print("Inner context")
        with lock:
            print("Innermost context")

## 3. Abstract Base Classes (ABC)

### 3.1 Creating Abstract Classes

In [ ]:
from abc import ABC, abstractmethod, ABCMeta
from collections.abc import Iterable, Container, Sized

# Basic abstract class
class Shape(ABC):
    @abstractmethod
    def area(self):
        pass
    
    @abstractmethod
    def perimeter(self):
        pass
    
    def description(self):
        return f"Shape with area {self.area():.2f} and perimeter {self.perimeter():.2f}"

# Concrete implementation
class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        import math
        return math.pi * self.radius ** 2
    
    def perimeter(self):
        import math
        return 2 * math.pi * self.radius

# Cannot instantiate abstract class
try:
    shape = Shape()  # This will raise TypeError
except TypeError as e:
    print(f"Error: {e}")

# Using concrete classes
rect = Rectangle(5, 3)
circle = Circle(4)

for shape in [rect, circle]:
    print(shape.description())

# Abstract properties
class Vehicle(ABC):
    @property
    @abstractmethod
    def max_speed(self):
        pass
    
    @property
    @abstractmethod
    def fuel_type(self):
        pass

class Car(Vehicle):
    @property
    def max_speed(self):
        return 200
    
    @property
    def fuel_type(self):
        return "Gasoline"

car = Car()
print(f"\nCar: max speed {car.max_speed} km/h, fuel: {car.fuel_type}")

### 3.2 Virtual Subclasses and Registration

In [ ]:
from abc import ABC, abstractmethod
import collections.abc

# Custom ABC with registration
class Drawable(ABC):
    @abstractmethod
    def draw(self):
        pass

# Register existing class as virtual subclass
@Drawable.register
class ExistingClass:
    def render(self):  # Note: different method name
        return "Rendering..."

# Check subclass
obj = ExistingClass()
print(f"Is ExistingClass a Drawable? {isinstance(obj, Drawable)}")
print(f"Is ExistingClass subclass of Drawable? {issubclass(ExistingClass, Drawable)}")

# Custom ABC with __subclasshook__
class Comparable(ABC):
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Comparable:
            # Check if class has required methods
            if any("__lt__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

# Any class with __lt__ is considered Comparable
class MyNumber:
    def __init__(self, value):
        self.value = value
    
    def __lt__(self, other):
        return self.value < other.value

num = MyNumber(5)
print(f"\nIs MyNumber Comparable? {isinstance(num, Comparable)}")

# Collections ABCs
class MyContainer:
    def __init__(self, items):
        self._items = list(items)
    
    def __contains__(self, item):
        return item in self._items
    
    def __iter__(self):
        return iter(self._items)
    
    def __len__(self):
        return len(self._items)

container = MyContainer([1, 2, 3])
print(f"\nIs Container? {isinstance(container, collections.abc.Container)}")
print(f"Is Iterable? {isinstance(container, collections.abc.Iterable)}")
print(f"Is Sized? {isinstance(container, collections.abc.Sized)}")

## 4. Advanced Type Hints

### 4.1 Generic Types and Type Variables

In [ ]:
from typing import (
    TypeVar, Generic, List, Dict, Optional, Union,
    Callable, Protocol, Literal, Final, TypedDict,
    overload, cast, get_type_hints
)
from typing import TYPE_CHECKING

# Type variables
T = TypeVar('T')
K = TypeVar('K')
V = TypeVar('V')

# Generic function
def first_element(items: List[T]) -> Optional[T]:
    return items[0] if items else None

# Test with different types
numbers = [1, 2, 3]
strings = ["a", "b", "c"]
print(f"First number: {first_element(numbers)}")
print(f"First string: {first_element(strings)}")

# Generic class
class Stack(Generic[T]):
    def __init__(self) -> None:
        self._items: List[T] = []
    
    def push(self, item: T) -> None:
        self._items.append(item)
    
    def pop(self) -> Optional[T]:
        return self._items.pop() if self._items else None
    
    def peek(self) -> Optional[T]:
        return self._items[-1] if self._items else None

# Type-specific stacks
int_stack: Stack[int] = Stack()
int_stack.push(1)
int_stack.push(2)
print(f"\nPopped: {int_stack.pop()}")

str_stack: Stack[str] = Stack()
str_stack.push("hello")
str_stack.push("world")
print(f"Peek: {str_stack.peek()}")

# Bounded type variables
from numbers import Number
TNum = TypeVar('TNum', bound=Number)

def sum_values(values: List[TNum]) -> TNum:
    return sum(values)

print(f"\nSum: {sum_values([1, 2, 3])}")
print(f"Sum: {sum_values([1.5, 2.5, 3.5])}")

### 4.2 Protocols and Structural Subtyping

In [ ]:
from typing import Protocol, runtime_checkable

# Define protocol
@runtime_checkable
class Drawable(Protocol):
    def draw(self) -> str:
        ...

@runtime_checkable
class Resizable(Protocol):
    def resize(self, factor: float) -> None:
        ...

# Classes that implement protocol (implicitly)
class Circle:
    def __init__(self, radius: float):
        self.radius = radius
    
    def draw(self) -> str:
        return f"Drawing circle with radius {self.radius}"
    
    def resize(self, factor: float) -> None:
        self.radius *= factor

class Square:
    def __init__(self, side: float):
        self.side = side
    
    def draw(self) -> str:
        return f"Drawing square with side {self.side}"

# Function accepting protocol
def render(shape: Drawable) -> None:
    print(shape.draw())

# Runtime checking
circle = Circle(5)
square = Square(4)

print(f"Circle is Drawable: {isinstance(circle, Drawable)}")
print(f"Circle is Resizable: {isinstance(circle, Resizable)}")
print(f"Square is Drawable: {isinstance(square, Drawable)}")
print(f"Square is Resizable: {isinstance(square, Resizable)}")

# Use with function
render(circle)
render(square)

# Protocol with properties
@runtime_checkable
class HasArea(Protocol):
    @property
    def area(self) -> float:
        ...

class Rectangle:
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height
    
    @property
    def area(self) -> float:
        return self.width * self.height

rect = Rectangle(3, 4)
print(f"\nRectangle has area: {isinstance(rect, HasArea)}")
print(f"Area: {rect.area}")

### 4.3 Advanced Type Hints Features

In [ ]:
from typing import (
    Literal, Final, TypedDict, overload,
    NewType, Type, ClassVar, Annotated
)
from dataclasses import dataclass

# Literal types
def set_mode(mode: Literal["read", "write", "append"]) -> None:
    print(f"Mode set to: {mode}")

set_mode("read")
set_mode("write")
# set_mode("delete")  # Type checker would flag this

# Final variables
MAX_SIZE: Final = 100
DEBUG_MODE: Final[bool] = True

class Config:
    VERSION: Final = "1.0.0"
    
    def __init__(self):
        self.setting: Final = "immutable"

# TypedDict
class PersonDict(TypedDict):
    name: str
    age: int
    email: str

person: PersonDict = {
    "name": "Alice",
    "age": 30,
    "email": "alice@example.com"
}

print(f"\nPerson: {person['name']}, {person['age']}")

# Function overloading
@overload
def process(data: int) -> str: ...

@overload
def process(data: str) -> int: ...

@overload
def process(data: list) -> dict: ...

def process(data):
    if isinstance(data, int):
        return str(data)
    elif isinstance(data, str):
        return len(data)
    elif isinstance(data, list):
        return {i: v for i, v in enumerate(data)}

print(f"\nProcess int: {process(42)}")
print(f"Process str: {process('hello')}")
print(f"Process list: {process([1, 2, 3])}")

# NewType
UserId = NewType('UserId', int)
ProductId = NewType('ProductId', int)

def get_user(user_id: UserId) -> str:
    return f"User {user_id}"

user_id = UserId(123)
print(f"\n{get_user(user_id)}")

# Annotated types
from typing import Annotated

PositiveInt = Annotated[int, "Must be positive"]
Email = Annotated[str, "Valid email format"]

@dataclass
class User:
    id: PositiveInt
    email: Email
    name: str

user = User(id=1, email="user@example.com", name="Bob")
print(f"User: {user}")

## 5. Slots and Memory Optimization

### 5.1 Using __slots__

In [ ]:
import sys

# Class without slots
class RegularClass:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

# Class with slots
class SlottedClass:
    __slots__ = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

# Compare memory usage
regular = RegularClass(1, 2, 3)
slotted = SlottedClass(1, 2, 3)

print("Memory comparison:")
print(f"Regular class instance: {sys.getsizeof(regular.__dict__)} bytes (dict)")
print(f"Slotted class instance: No __dict__")

# Regular class allows dynamic attributes
regular.new_attr = "dynamic"
print(f"\nRegular class with new attribute: {regular.new_attr}")

# Slotted class doesn't allow dynamic attributes
try:
    slotted.new_attr = "dynamic"
except AttributeError as e:
    print(f"Slotted class error: {e}")

# Slots with inheritance
class Base:
    __slots__ = ['a']

class Derived(Base):
    __slots__ = ['b']  # Adds to parent slots

derived = Derived()
derived.a = 1
derived.b = 2
print(f"\nDerived instance: a={derived.a}, b={derived.b}")

# Slots with descriptors
class OptimizedPoint:
    __slots__ = ['_x', '_y']
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
    
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        self._x = value
    
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value):
        self._y = value

point = OptimizedPoint(3, 4)
print(f"\nOptimized point: ({point.x}, {point.y})")

## 6. Weak References

### 6.1 Using weakref Module

In [ ]:
import weakref
import gc

# Basic weak reference
class MyClass:
    def __init__(self, value):
        self.value = value
    
    def __repr__(self):
        return f"MyClass({self.value})"

# Create object and weak reference
obj = MyClass(42)
weak_ref = weakref.ref(obj)

print(f"Original object: {obj}")
print(f"Via weak reference: {weak_ref()}")
print(f"Is alive: {weak_ref() is not None}")

# Delete strong reference
del obj
gc.collect()  # Force garbage collection

print(f"\nAfter deletion:")
print(f"Via weak reference: {weak_ref()}")
print(f"Is alive: {weak_ref() is not None}")

# WeakValueDictionary
cache = weakref.WeakValueDictionary()

class ExpensiveObject:
    def __init__(self, id):
        self.id = id
        print(f"Creating expensive object {id}")
    
    def __repr__(self):
        return f"ExpensiveObject({self.id})"

# Add to cache
obj1 = ExpensiveObject(1)
obj2 = ExpensiveObject(2)
cache['obj1'] = obj1
cache['obj2'] = obj2

print(f"\nCache contents: {list(cache.keys())}")

# Delete one object
del obj1
gc.collect()

print(f"After deleting obj1: {list(cache.keys())}")

# Weak reference with callback
def object_deleted(weak_ref):
    print(f"Object was deleted, weak ref: {weak_ref}")

obj3 = ExpensiveObject(3)
weak_ref_with_callback = weakref.ref(obj3, object_deleted)

print(f"\nDeleting obj3...")
del obj3
gc.collect()

# Proxy objects
obj4 = MyClass(100)
proxy = weakref.proxy(obj4)

print(f"\nProxy value: {proxy.value}")
proxy.value = 200
print(f"Modified via proxy: {obj4.value}")

## 7. Advanced Decorators

### 7.1 Class Decorators

In [ ]:
import functools
from typing import Any, Type

# Class decorator
def add_debug_repr(cls):
    """Add a debug __repr__ to class"""
    def __repr__(self):
        attrs = ', '.join(f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"{cls.__name__}({attrs})"
    
    cls.__repr__ = __repr__
    return cls

@add_debug_repr
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

point = Point(3, 4)
print(f"Point repr: {point}")

# Singleton decorator
def singleton(cls):
    """Make class a singleton"""
    instances = {}
    
    @functools.wraps(cls)
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    return get_instance

@singleton
class Database:
    def __init__(self):
        print("Creating database connection")
        self.connected = True

db1 = Database()
db2 = Database()  # Same instance
print(f"Same instance: {db1 is db2}")

# Property factory decorator
def auto_property(attr_name):
    """Create property with validation"""
    def decorator(cls):
        private_name = f'_{attr_name}'
        
        def getter(self):
            return getattr(self, private_name)
        
        def setter(self, value):
            if value < 0:
                raise ValueError(f"{attr_name} must be non-negative")
            setattr(self, private_name, value)
        
        setattr(cls, attr_name, property(getter, setter))
        return cls
    return decorator

@auto_property('age')
@auto_property('score')
class Student:
    def __init__(self, name, age, score):
        self.name = name
        self.age = age
        self.score = score

student = Student("Alice", 20, 95)
print(f"\nStudent age: {student.age}")

try:
    student.age = -5
except ValueError as e:
    print(f"Error: {e}")

### 7.2 Decorator Patterns

In [ ]:
import time
import functools
from typing import Callable, Any

# Parameterized decorator with optional arguments
def retry(max_attempts: int = 3, delay: float = 1.0):
    def decorator(func: Callable) -> Callable:
        @functools.wraps(func)
        def wrapper(*args, **kwargs) -> Any:
            last_exception = None
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    if attempt < max_attempts - 1:
                        print(f"Attempt {attempt + 1} failed, retrying...")
                        time.sleep(delay)
            raise last_exception
        return wrapper
    return decorator

@retry(max_attempts=3, delay=0.1)
def unreliable_function():
    import random
    if random.random() < 0.7:
        raise ValueError("Random failure")
    return "Success!"

try:
    result = unreliable_function()
    print(f"Result: {result}")
except ValueError as e:
    print(f"Failed after retries: {e}")

# Decorator with state
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"Call #{self.count} to {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def greet(name):
    return f"Hello, {name}!"

print(greet("Alice"))
print(greet("Bob"))
print(f"Total calls: {greet.count}")

# Method decorator
def validate_positive(func):
    @functools.wraps(func)
    def wrapper(self, value):
        if value <= 0:
            raise ValueError(f"Value must be positive, got {value}")
        return func(self, value)
    return wrapper

class Account:
    def __init__(self, balance):
        self.balance = balance
    
    @validate_positive
    def deposit(self, amount):
        self.balance += amount
        return self.balance
    
    @validate_positive
    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount
        return self.balance

account = Account(100)
print(f"\nDeposit: {account.deposit(50)}")

try:
    account.deposit(-10)
except ValueError as e:
    print(f"Error: {e}")

## 8. Advanced Iteration

### 8.1 Custom Iterators and Generators

In [ ]:
# Custom iterator class
class FibonacciIterator:
    def __init__(self, max_count):
        self.max_count = max_count
        self.count = 0
        self.current = 0
        self.next = 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.count >= self.max_count:
            raise StopIteration
        
        self.count += 1
        value = self.current
        self.current, self.next = self.next, self.current + self.next
        return value

# Using custom iterator
fib = FibonacciIterator(10)
print("Fibonacci sequence:")
for num in fib:
    print(num, end=' ')
print()

# Generator function
def fibonacci_generator(max_count):
    count, current, next_val = 0, 0, 1
    while count < max_count:
        yield current
        current, next_val = next_val, current + next_val
        count += 1

print("\nGenerator version:")
for num in fibonacci_generator(10):
    print(num, end=' ')
print()

# Coroutine with send
def running_average():
    total = 0
    count = 0
    average = None
    while True:
        value = yield average
        if value is not None:
            total += value
            count += 1
            average = total / count

# Using coroutine
avg = running_average()
next(avg)  # Prime the coroutine

print("\nRunning average:")
for value in [10, 20, 30, 40, 50]:
    result = avg.send(value)
    print(f"Added {value}, average: {result}")

# Generator with cleanup
def file_reader_generator(filename):
    try:
        file = open(filename, 'r')
        try:
            for line in file:
                yield line.strip()
        finally:
            file.close()
            print("File closed")
    except FileNotFoundError:
        print(f"File {filename} not found")
        return

# Infinite generator
def infinite_sequence():
    num = 0
    while True:
        yield num
        num += 1

# Use with islice
from itertools import islice
print("\nFirst 10 from infinite sequence:")
for num in islice(infinite_sequence(), 10):
    print(num, end=' ')
print()

## Module Summary

This module covered advanced Python features:

1. **Descriptors**: Custom attribute access control
2. **Context Managers**: Resource management and cleanup
3. **Abstract Base Classes**: Interface definitions and contracts
4. **Advanced Type Hints**: Generics, protocols, and type safety
5. **Slots**: Memory optimization for classes
6. **Weak References**: Avoiding circular references
7. **Advanced Decorators**: Class decorators and patterns
8. **Custom Iteration**: Iterators, generators, and coroutines

Key takeaways:
- Descriptors provide fine-grained control over attribute access
- Context managers ensure proper resource cleanup
- ABCs define interfaces and contracts
- Type hints improve code clarity and IDE support
- __slots__ can significantly reduce memory usage
- Weak references help with memory management
- Decorators enable clean separation of concerns
- Custom iterators provide flexible iteration patterns